In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/ajkam/schulich_data_science/main/Assignment%202/experiment_dataset.csv")

In [9]:
df.head(30)

,Unnamed: 0,Age,Location,Device,Variant,Time Spent,CTR
0,0,62,Location2,Device2,Control,13.928669,0.084776
1,1,18,Location1,Device1,Variant B,11.310518,0.096859
2,2,21,Location2,Device1,Variant B,24.842100,0.097630
3,3,21,Location1,Device3,Variant B,20.061300,0.109783
4,4,57,Location1,Device2,Variant B,34.495503,0.068579
5,5,27,Location3,Device1,Variant B,26.129246,0.149341
6,6,37,Location3,Device3,Variant B,20.525362,0.095788
7,7,39,Location2,Device1,Variant A,21.525217,0.149985
8,8,54,Location3,Device2,Control,21.910608,0.135535
9,9,41,Location1,Device2,Variant A,27.642788,0.137266


In [4]:
df.describe()

,Unnamed: 0,Age,Time Spent,CTR
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,40.715000,22.713345,0.109145
std,288.819436,13.655557,5.479292,0.022366
min,0.000000,18.000000,7.114806,0.012975
25%,249.750000,28.000000,19.216608,0.094286
50%,499.500000,41.000000,22.506707,0.108944
75%,749.250000,53.000000,26.255950,0.124238
max,999.000000,64.000000,39.395770,0.172728


In [7]:
print(df['Location'].unique())

['Location2' 'Location1' 'Location3']


In [6]:
print(df['Device'].unique())

['Device2' 'Device1' 'Device3']


In [5]:
print(df['Variant'].unique())

['Control' 'Variant B' 'Variant A']


In [10]:
Variant_A = df[df.Variant == 'Variant A']
Variant_A.describe()

,Unnamed: 0,Age,Time Spent,CTR
count,330.000000,330.000000,330.000000,330.000000
mean,503.448485,39.778788,24.805547,0.120269
std,292.544666,13.335836,4.964324,0.020109
min,7.000000,18.000000,7.800901,0.065667
25%,267.250000,28.000000,21.353587,0.107731
50%,492.000000,39.000000,24.955120,0.120203
75%,753.000000,51.000000,28.361732,0.134230
max,993.000000,64.000000,39.246751,0.172728


In [11]:
Variant_B = df[df.Variant == 'Variant B']
Variant_B.describe()

,Unnamed: 0,Age,Time Spent,CTR
count,330.000000,330.000000,330.000000,330.000000
mean,499.427273,41.466667,23.343783,0.108933
std,287.500859,13.931676,5.239965,0.020988
min,1.000000,18.000000,9.681810,0.041402
25%,236.500000,29.000000,19.941617,0.095024
50%,513.500000,42.000000,23.198123,0.109100
75%,748.750000,53.000000,26.604511,0.123861
max,999.000000,64.000000,39.395770,0.166768


In [12]:
Control = df[df.Variant == 'Control']
Control.describe()

,Unnamed: 0,Age,Time Spent,CTR
count,340.000000,340.000000,340.000000,340.000000
mean,495.738235,40.894118,20.070781,0.098554
std,287.254421,13.679236,5.124183,0.020523
min,0.000000,18.000000,7.114806,0.012975
25%,248.000000,28.000000,16.516020,0.084686
50%,484.000000,41.000000,20.337480,0.098895
75%,737.500000,53.000000,23.313397,0.111160
max,997.000000,64.000000,37.527368,0.168363


In [18]:
df.var()


C:\Users\Annuj\AppData\Local\Temp\ipykernel_3584\3059890634.py:1: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.var()


Unnamed: 0    83416.666667
Age             186.474249
Time Spent       30.022642
CTR               0.000500
dtype: float64

In [15]:
Variant_A.var()


C:\Users\Annuj\AppData\Local\Temp\ipykernel_3584\4116138393.py:1: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Variant_A.var()


Unnamed: 0    85582.381837
Age             177.844533
Time Spent       24.644508
CTR               0.000404
dtype: float64

In [16]:
Variant_B.var()


C:\Users\Annuj\AppData\Local\Temp\ipykernel_3584\1389376444.py:1: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Variant_B.var()


Unnamed: 0    82656.743935
Age             194.091591
Time Spent       27.457230
CTR               0.000440
dtype: float64

In [17]:
Control.var()

C:\Users\Annuj\AppData\Local\Temp\ipykernel_3584\859885082.py:1: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  Control.var()


Unnamed: 0    82515.102369
Age             187.121499
Time Spent       26.257248
CTR               0.000421
dtype: float64